In [117]:
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
import seaborn as sns

from itertools import product 
from sklearn.cluster import KMeans
import os
import sudoku as sudo

%matplotlib qt

In [118]:
n = 81
time = 60000
space = np.zeros((n,int(time)))

def U(x,gamma,I):
    tif = -1/gamma*np.log(1-(gamma/I))
    return I/gamma*(1-np.exp(-tif*x))

def U_inv(x,gamma,I):
    tif = -1/gamma*np.log(1-(gamma/I))
    return -1/tif*np.log(1-(gamma/I)*x)

def H(x,strength,gamma=1,I=1.008):
    temp = np.clip(U(x,gamma=gamma,I=I)+strength,0,None)
    fire = temp>=1
    temp = (temp<1)*temp
    return U_inv(temp,gamma=gamma,I=I),fire

inh = np.load("sudoku.npy")
exi = 1-inh
np.fill_diagonal(exi,0.0)
np.fill_diagonal(inh,0.0)

state = np.random.normal(size=n)%1

for i in range(int(time)):
    
    fire_time = np.min(1-state)   
    
    state = state + fire_time
    
    new_state = state + 0.01
    
    fire = new_state>=1
    
    while np.sum(fire)>0:

        new_state[fire] = 0.0
        
        coupling_exi = 0.0003*(np.matmul(exi,fire)>0)
        coupling_inh = 0.0004*np.matmul(inh,fire)
        
        total = coupling_exi-coupling_inh
        #print(total)
        
        state = new_state
        update = H(state,total)
        new_state,fire = np.clip(update[0],a_min=0,a_max=None),update[1]
        
    state = new_state
    space[:,i] = state
    if i%1000 == 0:
        print('Progress',round(i/int(time)*100,1),"%",end="\r")

In [119]:
rs =100
phase = space[:,::rs]-space[0,::rs]
phase[phase<0] = 1+phase[phase<0]

plt.figure()
cm = plt.cm.hsv(np.linspace(0.5,1,9))
for i in range(81):
    plt.plot(phase[i,:],'.',markersize=1)
plt.show()

In [120]:
df = pd.DataFrame(space[:,-1:])
model = KMeans(n_clusters=11)
pred = model.fit_predict(df)
order = np.argsort(pred)
#order = np.argsort(space[:,-1])
fig = plt.figure(figsize=(15,10))
g = sns.heatmap(space[order,-100:],yticklabels=2,xticklabels=100,cmap='viridis')
#g = sns.heatmap(space[:,-100:],yticklabels=2,xticklabels=100,cmap='viridis')
g.set_yticklabels(fig.axes[0].get_yticklabels(),rotation=0)
print()

In [93]:
from matplotlib.animation import FuncAnimation
import matplotlib as mpl

fig, ax = plt.subplots(figsize=(10,10))
ax.set(xlim=(-1.5, 1.5), ylim=(-1.5, 1.5))

def phasetopos(phi):
    return np.cos(2*np.pi*phi), np.sin(2*np.pi*phi)

temp = phasetopos(space[:,0])
data = ax.plot(temp[0],temp[1],'o',alpha=0.5)[0]

def animate(i):
    temp = phasetopos(space[:,i])
    data.set_xdata(temp[0])
    data.set_ydata(temp[1])

anim = FuncAnimation(
    fig, animate, interval=10, frames=time-1,repeat=False)
 
plt.draw()
plt.show()

In [116]:
for i in range(11):
    temp = np.arange(0,81,1,dtype=np.int32)[pred==i]
    sum_ = 0
    for j,k in product(temp,temp):
        sum_ += inh[j,k]
    print(sum_)

2
10
0
8
2
2
2
2
0
12
0


In [144]:
x = np.linspace(0,1,100)
plt.plot(x,U(x,1,1.008))
plt.plot(x,U_inv(x,1,1.008))
plt.plot(x,H(x,np.ones(x.shape)*0.05,1,1.008)[0])
plt.plot(x,x)

plt.show()

In [137]:
H(np.array([0,0.5]),[0.001,0.001],1,1.008)

(array([2.05231195e-04, 5.02315488e-01]), array([False, False]))